This is a vectorized version of the environment. How does it work? Simply put, instead of running a single environment, we're running them into batches (vectorization). At each time step, instead of formulating a single action $a$, we'll define it as a vector $a=[0, .., n_\text{envs}]$ where each entry corresponds to the action to be performed for an environment.

Thus, if you run 4 simulatenous environments, your observations space becomes (4, num_observations). Because we use a step size of 10 and have a total of 9 snesors, each observation will result into a space of (40,9) elements. The core idea is to speed up the inference and training time of the model instead of querying a single environment.

In our implementation, based on what model you decide, you should take as inputs the observations reshaped to your liking, and predict the actions $a$ from your policy.

In [1]:
%load_ext autoreload
%autoreload 2

In [55]:
import numpy as np
from student_client.student_gym_env_vectorized import create_student_gym_env_vectorized

env = create_student_gym_env_vectorized(
            num_envs=4,
            user_token='your_token'
        )

2026-02-18 20:17:26,941 - httpx - INFO - HTTP Request: GET http://127.0.0.1:8001/api/v1/version "HTTP/1.1 200 OK"
2026-02-18 20:17:26,942 - student_client.student_gym_env_vectorized - INFO - Client is up to date (version 0.2)
2026-02-18 20:17:26,948 - httpx - INFO - HTTP Request: POST http://127.0.0.1:8001/api/v1/session/create "HTTP/1.1 200 OK"
2026-02-18 20:17:26,949 - student_client.student_gym_env_vectorized - INFO - Created new session: f0d1f99e-aa61-4945-9cad-26c32b9f1c7e
2026-02-18 20:17:28,327 - httpx - INFO - HTTP Request: POST http://127.0.0.1:8001/api/v1/vectorized/episodes/create "HTTP/1.1 200 OK"
2026-02-18 20:17:28,328 - student_client.student_gym_env_vectorized - INFO - Created vectorized group with 4 episodes
2026-02-18 20:17:28,328 - student_client.student_gym_env_vectorized - INFO - Vectorized group ID: b0ade7b2-9abd-4029-8ca6-72c71ba7a146
2026-02-18 20:17:28,328 - student_client.student_gym_env_vectorized - INFO - StudentGymEnvVectorized initialized with 4 environmen

In [56]:
print(f"Environment created with {env.num_envs} parallel environments")
print(f"   Episode IDs: {env.episode_ids}")

# Reset all environments
print(f"\n🔄 Resetting all environments...")
observations, infos = env.reset()

print(f"   Observations shape: {observations.shape}")
print(f"   First observation: {observations[0]}")

Environment created with 4 parallel environments
   Episode IDs: ['2ec3bc39-fce0-44d0-9fea-4579be5bae56', 'a55b04ba-09da-4973-9fdf-afc78f790b9d', '05c8e819-9e6b-4c1b-a427-d4f0d1adc67c', 'a88ad325-ad8d-4ce7-b94c-f2ff339602e4']

🔄 Resetting all environments...


2026-02-18 20:17:32,259 - httpx - INFO - HTTP Request: POST http://127.0.0.1:8001/api/v1/episode/vectorized_reset "HTTP/1.1 200 OK"
2026-02-18 20:17:32,260 - student_client.student_gym_env_vectorized - INFO - All 4 environments reset successfully


   Observations shape: (4, 9)
   First observation: [7.9511169e+02 1.9362127e+04 3.3548248e+02 1.1194603e+03 3.7192494e-01
 1.3631862e+06 3.9559092e+03 0.0000000e+00 9.6814890e+00]


## Training / Iterations

Here you can iterate through the vectorized environments. You'll notice that actions are a vector where each entry corresponds to the associated environment in the vector.

In A), we automatically reset the envs that have terminated so you can continue for an indefinite amount of steps. As environments don't have the same length, they stop at different times, this helps you reset terminated episodes on the fly.

Tips:
- The step_size return many observations, should you feed each one-by-one in your model, or the full step_size=10 one? The choice is yours!
- There exists multiple ways of exploring the dataset

In [60]:
for step in range(40):

    # A) Check if any environments terminated
    terminated_envs = env.get_terminated_env_indices()
    if terminated_envs:

        print(f"   ⚠️  Environments {terminated_envs} terminated")
        reset_obs, reset_infos = env.reset_specific_envs(terminated_envs)
        for i, env_id in enumerate(terminated_envs):
            infos[env_id] = reset_infos[i] # reset previous info dict

        #break

    # Generate random actions for all environments
    actions = np.random.randint(1, 3, size=env.num_envs)
    #actions = np.asarray([0,0,0,0])

    print(f"\n   Step {step + 1}:")
    print(f"      Actions: {actions}")

    # Take step
    observations, rewards, terminateds, truncateds, infos = env.step(actions)

    print(f"      Rewards: {rewards}")
    print(f"      Terminated: {terminateds}")
    for i, obs in enumerate(observations):
        print(f"      Obs {i}: {obs.shape}")
    print(f"      Active environments: {env.get_active_count()}/{env.num_envs}")

    # Show filtered info (production mode)
    for i, info in enumerate(infos):
        print(f"      Env {i} info: {info}")

    #break

#env.close()

   ⚠️  Environments [1, 2, 3] terminated


2026-02-18 20:20:16,085 - httpx - INFO - HTTP Request: POST http://127.0.0.1:8001/api/v1/episode/vectorized_reset "HTTP/1.1 200 OK"



   Step 1:
      Actions: [1 1 2 2]


2026-02-18 20:20:17,772 - httpx - INFO - HTTP Request: POST http://127.0.0.1:8001/api/v1/episode/vectorized_step "HTTP/1.1 200 OK"


      Rewards: [-639.4836  -752.66034  136.75882  136.88219]
      Terminated: [False False  True  True]
      Obs 0: (10, 9)
      Obs 1: (10, 9)
      Obs 2: (1, 9)
      Obs 3: (1, 9)
      Active environments: 2/4
      Env 0 info: {'step': 30, 'terminated': False, 'truncated': False}
      Env 1 info: {'step': 10, 'terminated': False, 'truncated': False}
      Env 2 info: {'step': 1, 'terminated': True, 'truncated': False}
      Env 3 info: {'step': 1, 'terminated': True, 'truncated': False}
   ⚠️  Environments [2, 3] terminated


...

2026-02-18 20:20:23,183 - httpx - INFO - HTTP Request: POST http://127.0.0.1:8001/api/v1/episode/vectorized_reset "HTTP/1.1 200 OK"



   Step 3:
      Actions: [1 2 2 2]


2026-02-18 20:20:24,479 - httpx - INFO - HTTP Request: POST http://127.0.0.1:8001/api/v1/episode/vectorized_step "HTTP/1.1 200 OK"


      Rewards: [-594.2397   136.04633  115.29334  137.94272]
      Terminated: [False  True  True  True]
      Obs 0: (10, 9)
      Obs 1: (1, 9)
      Obs 2: (1, 9)
      Obs 3: (1, 9)
      Active environments: 1/4
      Env 0 info: {'step': 50, 'terminated': False, 'truncated': False}
      Env 1 info: {'step': 1, 'terminated': True, 'truncated': False}
      Env 2 info: {'step': 11, 'terminated': True, 'truncated': False}
      Env 3 info: {'step': 1, 'terminated': True, 'truncated': False}
   ⚠️  Environments [1, 2, 3] terminated


KeyboardInterrupt: 

In [46]:
for obs_env in observations:
    print(obs_env.shape)

(1, 9)
(1, 9)
(1, 9)
(1, 9)


In [51]:
observations

[array([[7.9119489e+02, 1.9313773e+04, 3.3493365e+02, 1.1183016e+03,
         3.7162682e-01, 1.3504325e+06, 3.9524648e+03, 0.0000000e+00,
         9.1000004e+00]], dtype=float32),
 array([[7.9269122e+02, 1.9191648e+04, 3.3450665e+02, 1.1146975e+03,
         3.7097099e-01, 1.3704700e+06, 3.9508916e+03, 0.0000000e+00,
         9.2736034e+00]], dtype=float32),
 array([[7.9539990e+02, 1.9358600e+04, 3.3558124e+02, 1.1196388e+03,
         3.7197024e-01, 1.3640279e+06, 3.9565410e+03, 0.0000000e+00,
         9.4699183e+00]], dtype=float32),
 array([[7.9565344e+02, 1.9362465e+04, 3.3514899e+02, 1.1180753e+03,
         3.7165123e-01, 1.3695479e+06, 3.9540964e+03, 0.0000000e+00,
         9.7564869e+00]], dtype=float32)]